In [4]:
import cvxpy as cp
import numpy as np

In [122]:
def skew_matrix(vectors):
    # assert vectors.shape[-1] == 1, vectors.shape
    skew = np.zeros( vectors.shape[:-1] + (3,3) )

    # print()
    # print( f"{vectors.shape=}")
    # print( f"{skew.shape=}")

    skew[..., 0, 1] = -vectors[..., 2]
    skew[..., 1, 2] = -vectors[..., 0]
    skew[..., 2, 0] = -vectors[..., 1]
    skew[..., 1, 0] =  vectors[..., 2]
    skew[..., 2, 1] =  vectors[..., 0]
    skew[..., 0, 2] =  vectors[..., 1]
    
    return skew

def example_rotation_transform(normals):
    #hopefully no one will try grabing directly under or above
    global_z_axis = np.array([0,0,1])

    # print( f"{global_z_axis.shape=}")
    # print( f"{skew_matrix(global_z_axis).shape=}")
    # print( f"{normals.shape=}")

    #  n,3, 1      3, 3                       n, 3, 1
    local_x = skew_matrix(global_z_axis) @ normals[..., None]
    # print(  f"{local_x.shape=}")

    #  n,3,1         n,3,3              n,3,1
    local_y = skew_matrix(normals) @ local_x

    # print(local_x.shape)
    # print(local_y.shape)
    # print(normals.shape)

    rotations = np.stack([ local_x, local_y, normals[..., None] ], axis=-1)[...,0,:]
    return rotations

In [132]:
def get_forces(positions, normals, target_force):
    mu = 0.5


    n, _ = positions.shape
    assert normals.shape == (n, 3)
    assert target_force.shape == (3,)

    weight = 0*np.array([0,0,-1])

    F = cp.Variable( (n, 3) )
    constraints = []

    normals = normals /  np.linalg.norm( normals, axis=-1, keepdims=True)

    total_force = np.zeros((3))
    total_torque = np.zeros((3))

    for pos,norm,f in zip(positions, normals, F):
        q = example_rotation_transform(norm)

        total_force += q @ f
        total_torque += skew_matrix(pos) @ q @ f

    total_force += weight
    constraints.append( total_force == target_force )
    constraints.append( total_torque == 0 )

    friction_cone = cp.norm(F[:,:2], axis=1) <= mu * F[:,2]
    constraints.append( friction_cone )


    force_magnitudes = cp.norm(F, axis=1)
    prob = cp.Problem(cp.Minimize(cp.max(force_magnitudes)), constraints)
    prob.solve()

    return F.value


In [134]:

positions = np.array([ [1,1,0],
                       [-1,1,0],
                       [-1,-1,0],
                       [1,-1,0],
                           ])


normals = np.array([ [-1,-1,0],
                       [1,-1,0],
                       [1,1,0],
                       [-1,1,0],
                           ])

target_force = np.array([ 0.1,0,0 ] )


get_forces(positions, normals, target_force)

array([[ 6.25341632e-13,  0.00000000e+00, -8.60034488e-11],
       [ 2.35702259e-02,  0.00000000e+00,  4.71404521e-02],
       [-2.35702259e-02,  0.00000000e+00,  4.71404521e-02],
       [-6.25822545e-13,  0.00000000e+00, -8.60068674e-11]])

In [ ]:

# N,3 
# positions = np.array()

# N,3 
# normals = np.array()

# 3 
# target_force = np.array()

def get_forces(positions, normals, target_force):
    if len(positions.shape) == 2:
        positions = positions[..., None]

    if len(normals.shape) == 2:
        normals = normals[..., None]

    if len(target_force.shape) == 1:
        target_force = target_force[..., None]

    n, _, _ = positions.shape
    weight = 0*np.array([0,0,-1])[:,None]
    
    mu = 0.5
    
    F = cp.Variable( (n, 3) )
    constraints = []
    
    normals = normals /  np.linalg.norm( normals, axis=-2, keepdims=True)

    # local to global, n, 3, 3
    Q = example_rotation_transform(normals)
    
    # n, 3, 3
    S = skew_matrix(positions)

    total_force = np.zeros((3))
    for q,f in zip(Q,F):
        total_force += q @ f

    total_force += weight - target_force

    # # 3, 1
    # total_force = cp.sum(Q @ f + weight - target_force, axis = 0)
    constraints.append( total_force == 0 )


    total_torque = np.zeros((3))
    for s,q,f in zip(S,Q,F):
        total_torque += s @ q @ f
    
    # 3, 1
    # total_torque = cp.sum(S @ Q @ f, axis = 0)
    constraints.append( total_torque == 0 )
    
    friction_cone = cp.norm(f[:,:2, 0], axis=-1) < mu * f[:,2, 0]
    constraints.append( friction_cone )
    
    
    force_magnitudes = cp.norm(f[:,:, 0], axis=-1)
    prob = cp.Problem(cp.Minimize(cp.max(force_magnitudes)), constraints)
    prob.solve()

    return f.value

